In [17]:
library(dplyr)
library(tidyr)

# Merging metadata tables

In [18]:
m <- '~/data/awtp2_16S/metadata.tsv'
fgl_data <- '~/data/awtp2_16S/19_0708_KNLK_SD_DNA.csv'
mwq <- '~/data/awtp2_16S/mwq_022120.csv'

In [19]:
#what should it all be called?
NTB<- "90%_conditioned/10%_conventional" #90/10 blends
half<- "45%_conditioned/55%_conventional" #45/55 blends
dpwf<- "100%_conditioned"   #source & merge
mwtp_s<-"100%_conventional" #source & merge

mwtp<- "100%_conventional_water" # influent composition
all_pure<-"100%_conditioned_water"  #influent composition

all_conv<- "100%_conventional" #location_code
dpwf_s<- "100%_conditioned" #location_code

In [20]:
seq_data=read.table(m, header=TRUE, sep="\t")
seq_data$sample_date <- as.Date(seq_data$sample_date, "%m/%d/%Y")
seq_data$location_code<- gsub("Loop_MWTP_inf",all_conv,seq_data$location_code)
seq_data$location_code<- gsub("Loop_DPWF_inf",dpwf_s,seq_data$location_code)
seq_data$location_code<- gsub("DWDS_282B","DWDS_282",seq_data$location_code)
seq_data$location_code<- gsub("WW_3ary","WW_tertiary",seq_data$location_code)

#label all samples as "N" for non-experimental
seq_data$exp_merge<-"N"
#label experimental samples (negative controls and positive controls) as experimental (because these rows won't have water quality data to merge)
seq_data[((seq_data$location_code=="negative_control")| (seq_data$location_code=="positive_control"))|(seq_data$location_code=="field_blank"),]$exp_merge<-"Y"

head(seq_data)

id,id_full,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,location_code,sample_type,sampler_name,Ext_batch,Extraction_Date,Ext_num_unique,Microconcentrated.,DNA_Ext_conc_ngperuL,kit,exp_merge
<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<chr>
SD001,KNLK_SD001,21320,TACGAGAC,GAGCTCAT,1,D2,Y,EXP_0_field_bulk_9_NA_19,2018-04-23,⋯,field_blank,DEUF,Lauren_Kennedy,19,9/8/2018,183,Y,0.126,PowerSoil Pro,Y
SD002,KNLK_SD002,43792,CTACTATA,CTCTAGAG,1,A3,Y,EXP_0_extraction_NA_NA_NA_11,NA,⋯,negative_control,NA,NA,11,7/7/2018,94,Y,0.226,PowerSoil Pro,Y
SD003,KNLK_SD003,52728,CGTTACTA,CTCTAGAG,1,B3,Y,DWDS_F_utility_bulk_9_A_12,2018-03-28,⋯,Loop_MWTP_inf,DEUF,Lauren_Kennedy,12,7/9/2018,99,Y,0.252,PowerSoil Pro,N
SD004,KNLK_SD004,35359,AGAGTCAC,CTCTAGAG,1,C3,Y,SDS_F_5_biofilm_9_NA_8,2018-04-09,⋯,Loop_5,biofilm,Lauren_Kennedy,8,7/6/2018,51,Y,0.302,PowerSoil Pro,N
SD005,KNLK_SD005,35380,TACGAGAC,CTCTAGAG,1,D3,Y,DWDS_F_328_bulk_5_A_13,2018-04-19,⋯,DWDS_328,DEUF,Lauren_Kennedy,13,8/23/2018,116,Y,0.307,PowerSoil Pro,N
SD006,KNLK_SD006,30734,TACGAGAC,CATGAGGA,1,D5,Y,AWTP_2_RO2_biofilm_sep_1_A_RO1_A57,2018-01-10,⋯,RO_BF_separator,biofilm,Rose_Kantor,RO1,9/13/2018,A57,Y,1.17,Powersoil Pro,N


In [5]:
ext_data <- read.csv(fgl_data, header=TRUE, row.names = NULL)
head(ext_data)

id,buffer,volume_uL,Avg.Size,ng.uL,nM,uL_for_200ng
<fct>,<fct>,<int>,<int>,<dbl>,<dbl>,<dbl>
KNLK_SD185,18 mM TAE bufer,15,385,0.1300,0.5556,359.971202
KNLK_SD003,20 mM tris-HCl with 0.1 mM EDTA,50,393,10.0087,41.8977,4.773532
KNLK_SD004,20 mM tris-HCl with 0.1 mM EDTA,50,390,7.1627,30.2149,6.619251
KNLK_SD005,20 mM tris-HCl with 0.1 mM EDTA,50,390,19.2228,81.0844,2.466566
KNLK_SD006,20 mM tris-HCl with 0.1 mM EDTA,50,393,12.7806,53.5584,3.734242
KNLK_SD007,20 mM tris-HCl with 0.1 mM EDTA,50,396,36.0204,149.5023,1.337772


In [6]:
wq_data <- read.csv(mwq, header=TRUE, row.names = NULL)
wq_data <- select(wq_data, select=-X)
wq_data$sample_date <- as.Date(wq_data$sample_date, "%Y-%m-%d")
wq_data$location_code <- as.character(wq_data$location_code)
wq_data$location_code<- gsub("WW_Tertiary","WW_tertiary",wq_data$location_code)
wq_data$location_code<- gsub("DPWF_inf",'cond_perm',wq_data$location_code)
wq_data$sample_type<- gsub("bulkwater_grab",'DEUF',wq_data$sample_type)
wq_data<-subset(wq_data, select=-c(project_type, merge_id))
wq_data=subset(wq_data, is.na(wq_data$experimental_detail))
wq_data$exp_merge<-"N" #sample date and location code shared by some samples so making it easier to merge seq_data

In [7]:
unique(wq_data$sample_type)

[1] "DEUF"

In [8]:
unique(seq_data$sample_type)

[1] DEUF    <NA>    biofilm
Levels: biofilm DEUF

In [10]:
rownames(table_data) <- table_data$id_full

# convert to correct class
table_data$batch_sample_date <- as.Date(table_data$batch_sample_date, "%m/%d/%Y")
table_data$sample_type <- as.factor(table_data$sample_type)
table_data$location_code <- as.factor(table_data$location_code)
table_data$Extraction_Date <- as.Date(table_data$Extraction_Date, "%m/%d/%Y")
table_data$DNA_Ext_conc_ngperuL <- as.character(table_data$DNA_Ext_conc_ngperuL) # factor to character
table_data$DNA_Ext_conc_ngperuL <- as.numeric(table_data$DNA_Ext_conc_ngperuL) # character to numeric/decimal

#No wq data taken for pipe loops batch end on 10/23 or 2/19

Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”

In [9]:
table_data <- merge(seq_data, ext_data, by.x='id_full', by.y='id')
table_data <- left_join(table_data,wq_data)

Joining, by = c("sample_date", "location_code", "sample_type", "exp_merge")
Warning message:
“Column `sample_type` joining factor and character vector, coercing into character vector”

In [11]:
# check that it worked
table_data[25:35,]

,id_full,id,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,base_filename,FCM_notes,avg_cells_per_mL_gmean_SGPI,avg_cells_per_mL_gstd_SGPI,intra_ATP_gmean_nM,total_ATP_gmean_nM,extra_ATP_gmean_nM,intra_ATP_gstd_nM,total_ATP_gstd_nM,extra_ATP_gstd_nM
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
KNLK_SD028,KNLK_SD028,SD028,35200,CTACTATA,TAGACTGA,1,A11,Y,AWTP_2_BAC_bulk_5_A_2,2017-12-14,⋯,17_1214_DPWF_3_GAC_SGPI_10x_,NA,475688.4,1.691495,0.11222116,0.13538441,0.023163254,1.450614,1.011282,1.039995
KNLK_SD029,KNLK_SD029,SD029,30964,CGTTACTA,TAGACTGA,1,B11,Y,AWTP_3_inf_bulk_1_A_1,2018-03-13,⋯,18_0313_ECAWPC_Influent_SGPI_10x_2,NA,12146298.6,1.074069,NA,NA,NA,NA,NA,NA
KNLK_SD030,KNLK_SD030,SD030,38480,AGAGTCAC,TAGACTGA,1,C11,Y,AWTP_2_inf_bulk_4_A_1,2017-11-14,⋯,17_1114_DPWF_1_Influent_SGPI_100x_,NA,5461380.2,1.014661,0.55397549,0.65390039,0.099924893,1.591198,1.224577,1.016032
KNLK_SD031,KNLK_SD031,SD031,37100,TACGAGAC,TAGACTGA,1,D11,Y,AWTP_2_inf_bulk_5_A_1,2017-12-14,⋯,17_1214_DPWF_1_Influent_SGPI_100x_,NA,6660896.6,1.070527,0.63648601,0.78860455,0.152118540,1.494909,1.081167,1.032391
KNLK_SD032,KNLK_SD032,SD032,23905,ACGTCTCG,TAGACTGA,1,E11,Y,AWTP_2_RO2_biofilm_ret_scrape_1_B,2018-01-10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD033,KNLK_SD033,SD033,41694,TCGACGAG,TAGACTGA,1,F11,Y,AWTP_2_inf_bulk_3_A_1,2017-10-10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD034,KNLK_SD034,SD034,34252,GATCGTGT,TAGACTGA,1,G11,N,AWTP_2_cond_bulk_7_A_3,2018-03-01,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD035,KNLK_SD035,SD035,31665,GTCAGATA,TAGACTGA,1,H11,N,AWTP_2_inf_bulk_1_A_1,2017-09-14,⋯,17_0914_DPWF_1_Influent_SGPI_100x_,NA,9227256.9,1.072321,0.36737778,0.67470901,0.307331225,1.530134,1.096372,1.067370
KNLK_SD036,KNLK_SD036,SD036,36007,CTACTATA,GTATGACG,1,A12,N,AWTP_2_BAC_bulk_1_A_2,2017-09-14,⋯,17_0914_DPWF_3_GAC_SGPI_10x_,NA,709814.7,1.026051,0.03903449,0.06693995,0.027905461,1.431873,1.018637,1.006299


In [76]:
table_data <- table_data[,c("id",setdiff(names(table_data),"id"))] # move id column to be first on the left
head(table_data)
#write.table(table_data, '~/data/awtp2_16S/metadata_final.tsv', sep="\t", quote=FALSE, row.names=FALSE)

,id,id_full,count_raw_reads,barcode_forward,barcode_reverse,plate,well,metagenomic_sample,sample_code_full_and_batch,sample_date,⋯,base_filename,FCM_notes,avg_cells_per_mL_gmean_SGPI,avg_cells_per_mL_gstd_SGPI,intra_ATP_gmean_nM,total_ATP_gmean_nM,extra_ATP_gmean_nM,intra_ATP_gstd_nM,total_ATP_gstd_nM,extra_ATP_gstd_nM
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<date>,⋯,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
KNLK_SD003,SD003,KNLK_SD003,52728,CGTTACTA,CTCTAGAG,1,B3,Y,DWDS_F_utility_bulk_9_A_12,2018-03-28,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD004,SD004,KNLK_SD004,35359,AGAGTCAC,CTCTAGAG,1,C3,Y,SDS_F_5_biofilm_9_NA_8,2018-04-09,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD005,SD005,KNLK_SD005,35380,TACGAGAC,CTCTAGAG,1,D3,Y,DWDS_F_328_bulk_5_A_13,2018-04-19,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD006,SD006,KNLK_SD006,30734,TACGAGAC,CATGAGGA,1,D5,Y,AWTP_2_RO2_biofilm_sep_1_A_RO1_A57,2018-01-10,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD007,SD007,KNLK_SD007,35135,CTACTATA,AGTCTAGA,1,A6,Y,AWTP_3_MF_tank_bulk_1_A_18,2018-03-13,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
KNLK_SD008,SD008,KNLK_SD008,16805,CGTTACTA,AGTCTAGA,1,B6,Y,DWDS_F_328_bulk_1_A_12,2018-02-14,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Add to phyloseq object

In [ ]:
ps <- readRDS("~/data/awtp2_16S/ps_clean_filt.rds"

In [ ]:
# Add metadata
metadata_complete <- read.table('~/data/awtp2_16S/metadata_final.tsv', sep="\t", header=TRUE, row.names=1)
metadata_complete$location_code<- gsub("WW_tertiary", "WW_3ary", metadata_complete$location_code)

#reset order of levels
location_levels = c(
    'field_blank',
    'RO_BF_control',
    'negative_control',
    'WW_3ary',
    'BAC',
    'MF_combined',
    'MF_tank',
    'RO_BF_retentate_scrapings',
    'RO_BF_separator',
    'RO_combined',
    'RO_3stage', 'RO_2stage', 'RO',
    'cond_perm', 'site_ut', 'Loop_1', 'Loop_2', 'Loop_3', 'Loop_4', 'Loop_5',
    'DWDS_328', 'DWDS_286', 'DWDS_282B')
metadata_complete$location_code <- factor(metadata_complete$location_code, levels=location_levels)

head(metadata_complete)

In [ ]:
sample_data(ps) <- metadata_complete

In [ ]:
#change ASV names to short codes, store DNA seqs in ref_seq object in phyloseq
dna <- Biostrings::DNAStringSet(taxa_names(ps))
names(dna) <- taxa_names(ps)
ps <- merge_phyloseq(ps, dna)
taxa_names(ps) <- paste0("ASV", seq(ntaxa(ps)))
ps

In [ ]:
saveRDS(ps, '~/data/awtp2_16S/ps_clean_filt_metadata.rds')

# Update AWTP names

In [ ]:
# select only samples of interest
ps_dpwf <- subset_samples(ps, subset = project_type %in% c("DPWF", "ECAWPC"))
ps_dpwf <- filter_taxa(ps_dpwf, function(x) sum(x) > 0, TRUE) #keep only seqs that had reads

In [ ]:
sample_data(ps_dpwf)$location_gen <- sample_data(ps_dpwf)$location_code
                                            
new_levels <- c('WW_3ary',
'BAC',
'MF_combined',
'MF_tank',
'MF',
'RO_BF_retentate_scrapings',
'RO_BF_separator',
'RO_combined',
'RO_3stage',
'RO_2stage',
'RO')

sample_data(ps_dpwf)$location_gen <- factor(sample_data(ps_dpwf)$location_gen, levels=new_levels)

In [ ]:
# change factor levels to condense down to simple location
sample_data(ps_dpwf)[sample_data(ps_dpwf)$location_code %in% c("MF_tank", "MF_combined"),]$location_gen <- "MF"
sample_data(ps_dpwf)[sample_data(ps_dpwf)$location_code %in% c("RO", "RO_combined", "RO_3stage", "RO_2stage"),]$location_gen <- "RO"